The World Register of Marine Species (WoRMS) is another taxonomic authority we use in our work. In the case of the FWS work plan species, we mostly rely on ITIS as that is the taxonomic authority most used by FWS, and ITIS TSNs are determined/declared for most species. This notebook follows the ITIS caching process to retrieve any unmatched names and try them against WoRMS using the worms module of the bispy package.

In [1]:
import requests
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed

worms = bispy.worms.Worms()
bis_utils = bispy.bis.Utils()

In [2]:
with open("cache/itis.json", "r") as f:
    itis_cache = json.loads(f.read())

In [3]:
#Unwind the ITIS URLs into names
unmatched_itis_names = [i["Processing Metadata"]["Detailed Results"][0]["Exact Match Fail"].replace("http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:", "").replace("\\%20", " ") for i in itis_cache if i["Processing Metadata"]["Summary Result"] == "Not Matched"]


In [4]:
unmatched_itis_names

['Oreohelix n. sp. 1',
 'Monadenia fidelis minor',
 'Vertigo sp.',
 'Eurycea sp.',
 'Ictalurus sp.',
 'astylis species',
 'Lycaena ferrisi',
 'Percina kusha',
 'Etheostoma maydeni',
 'Cottus sp.',
 'Boltonia montana',
 'Hemileuca spp.',
 'Pyrgulopsis hamlinensis',
 'Pyrgulopsis anguina',
 'Pyrgulopsis saxatilis',
 'Euphilotes ancilla cryptica',
 'Pyrgulopsis peculiaris',
 'Pyrgulopsis coloradensis',
 'Pyrgulopsis lata',
 'Pyrgulopsis fausta',
 'Pyrgulopsis breviloba',
 'Pyrgulopsis planulata',
 'Pyrgulopsis marcida',
 'Pyrgulopsis hubbsi',
 'Pyrgulopsis sublata',
 'Pyrgulopsis landyei',
 'Pyrgulopsis neritella',
 'Pyrgulopsis serrata',
 'Pyrgulopsis turbatrix',
 'Pyrgulopsis sulcata',
 'Pyrgulopsis deaconi',
 'Pyrgulopsis sterilis',
 'Pyrgulopsis orbiculata',
 'Pyrgulopsis sathos']

In [5]:
%%time
# Use joblib to run multiple searches for WoRMS species in parallel via species names
worms_result = Parallel(n_jobs=8)(delayed(worms.search)(name) for name in unmatched_itis_names)

CPU times: user 129 ms, sys: 66.3 ms, total: 195 ms
Wall time: 9.66 s


In [6]:
# We're just going to use the exact match records from WoRMS at this point
worms_cache = [w for w in worms_result if w["Processing Metadata"]["Summary Result"] == "Exact Match"]

In [7]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/worms.json", worms_cache))

{'Doc Cache File': 'cache/worms.json',
 'Document Number 11': {'Processing Metadata': {'Date Processed': '2019-07-03T21:41:30.652701',
   'Search URL': 'http://www.marinespecies.org/rest/AphiaRecordsByName/Pyrgulopsis planulata?like=false&marine_only=false&offset=1',
   'Status': 'Error',
   'Summary Result': 'Exact Match'},
  'wormsData': [{'AphiaID': 1349194,
    'authority': 'Hershler, 1998',
    'citation': 'MolluscaBase (2019). MolluscaBase. Pyrgulopsis planulata Hershler, 1998. Accessed through: World Register of Marine Species at: http://www.marinespecies.org/aphia.php?p=taxdetails&id=1349194 on 2019-07-03',
    'class': 'Gastropoda',
    'family': 'Hydrobiidae',
    'genus': 'Pyrgulopsis',
    'isBrackish': 0,
    'isExtinct': 0,
    'isFreshwater': 1,
    'isMarine': 0,
    'isTerrestrial': 0,
    'kingdom': 'Animalia',
    'lsid': 'urn:lsid:marinespecies.org:taxname:1349194',
    'match_type': 'exact',
    'modified': '2019-06-02T22:12:05.937Z',
    'order': 'Littorinimorpha'